<a href="https://colab.research.google.com/github/mpugna/Advanced-Deep-Trading/blob/master/CME_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import numpy as np
import pandas as pd
import json
import datetime as dt
import calendar
from pandas.tseries.offsets import BMonthEnd
from typing import Union
import time
from random import randint

In [0]:
def month_str_to_num(month: str) -> int:
    """
    Convert a month abbreviated with its first three letters
    to the corresponding number.
    """
    conversion = {
        'jan': 1,
        'feb': 2,
        'mar': 3,
        'apr': 4,
        'may': 5,
        'jun': 6,
        'jul': 7,
        'aug': 8,
        'sep': 9,
        'oct': 10,
        'nov': 11,
        'dec': 12,
    }
    if month.lower() in conversion.keys():
        return conversion[month.lower()]
    else:
        raise ValueError('Cannot convert month <{}>'.format(month))
        
def str_date_to_datetime(x: str) -> dt.date:
    x = ' '.join(x.split())  # collapse whitespaces
    x = x.strip()  # remove trailing/leading whitespaces
    x = x.lower()
    three_letters_month, two_digits_year = x.split(' ')
    month = month_str_to_num(three_letters_month)
    year = 2000 + int(two_digits_year)
    return third_friday_of_month(year, month)

def third_friday_of_month(year: int, month: int) -> dt.date:
    """
    Get the date corresponding to the third friday of the year/month specified.

    example: calling option_expiration(2019, 10)
             returns datetime.date(2019,10,18)
    """
    day = 21 - (calendar.weekday(year, month, 1) + 2) % 7
    return dt.date(year, month, day)


def last_business_day_of_month(year: int, month: int) -> dt.date:
    return BMonthEnd().rollforward(dt.date(year, month, 1)).date()


def day_of_week(year: int, month: int, week: int, day_of_week: int) -> dt.date:
    n_weeks = 0
    for days_in_week in calendar.monthcalendar(year, month):
      if days_in_week[day_of_week] != 0:
        n_weeks += 1
        if n_weeks == week:
          return dt.date(year, month, days_in_week[day_of_week])

    return None

def monday_of_week(year: int, month: int, week: int) -> dt.date:
    return day_of_week(year, month, week, 0)

def wednesday_of_week(year: int, month: int, week: int) -> dt.date:
    return day_of_week(year, month, week, 2)

def friday_of_week(year: int, month: int, week: int) -> dt.date:
    return day_of_week(year, month, week, 4)  

In [0]:
def get_options_details(proxies: dict=None) -> dict:
    """
    For each options group retrieve the following informations:
        - expiry label (e.g.- "Mar 2020")
        - expiry month (e.g. 3)
        - expiry year (e.g. 2020)
        - expiry code (e.g. "H0")
        - product id (e.g. 138)
        - contract_id (e.g. "EZH20")
        - trade_date (e.g. datetime.datetime(2020,3,20))
    """
    url = 'https://www.cmegroup.com/CmeWS/mvc/Settlements/Options/TradeDateAndExpirations/133'
    page = requests.get(url, proxies=proxies if proxies is not None else dict(), verify=False)
    if page.status_code != 200:
        raise ValueError(f'Cannot retrieve the page {url}. HTTP status code = {page.status_code}')

    table = dict()
    # For each options group in the returned json data extract
    # the relevant data
    for subset in page.json():
        name = subset['name']  # group name, e.g. "E-mini S&P 500 Options"
        
        table[name] = dict()
        # Get the details of the current options group
        expirations = subset['expirations']
        for expiry in expirations:
            label = expiry['label']  # the expiry as a string, e.g. "Mar 2020"

            table[name][label] = dict()
            table[name][label]['month'] = expiry['expiration']['month']  # e.g. 3
            table[name][label]['year'] = expiry['expiration']['year']    # e.g. 2020
            table[name][label]['code'] = expiry['expiration']['code']    # e.g. "H0"
            #table[name][label]['two_digits_code'] = expiry['expiration']['twoDigitsCode']  # e.g. "H20"
            table[name][label]['product_id'] = expiry['productId']  # e.g. 138
            table[name][label]['contract_id'] = expiry['contractId']  # e.g. "EZH20"
            table[name][label]['trade_date'] = pd.to_datetime(expiry['lastTradeDate']['dateOnlyLongFormat']).date()

    return table

In [0]:
def retrieve_options_chain(table: dict, proxies: dict=None) -> dict:
    """
    Given a dictionary returned from the get_options_details() routine
    retrieve the complete option chain for each expiry date.
    """
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=2.)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    for name, details in table.items():
        print(name)
        for label, data in details.items():
            product_id = data['product_id']
            contract_id = data['contract_id']
            code = data['code']
            trade_date = data['trade_date']
            print('   ', label)
            
            url = ''.join(
                (
                    f'https://www.cmegroup.com/CmeWS/mvc/Settlements/Options/Settlements//{product_id}/OOF?',
                    f'monthYear={contract_id}&strategy=DEFAULT&optionProductId={product_id}&', 
                    f'optionExpiration={product_id}-{code}&',
                    'tradeDate={:02d}%2F{:02d}%2F{:2d}&pageSize=1000'.format(trade_date.month, trade_date.day, trade_date.year),
                )
            )
            #time.sleep(randint(5,60))   # randomly wait between 5 and 60 seconds
            page = session.get(url, proxies=proxies if proxies is not None else dict(), verify=False)
            if page.status_code == 200:
                td = pd.to_datetime(page.json()['tradeDate'], dayfirst=True).date()
                prices = pd.DataFrame(page.json()['settlements'])
                prices = prices[prices['strike']!='Total']  # Remove the row with the totals
                prices = prices[prices['settle']!='CAB']
                prices['strike'] = pd.to_numeric(prices['strike'].str.replace(',', ''))
                prices['settle'] = pd.to_numeric(prices['settle'].str.replace(',', ''))
                prices['volume'] = pd.to_numeric(prices['volume'].str.replace(',', ''))
                prices['openInterest'] = pd.to_numeric(prices['openInterest'].str.replace(',', ''))
                prices['last'] = prices['last'].apply(lambda x: x.strip().lower().replace('a', '').replace('b', ''))
                prices['last'] = prices['last'].str.strip().replace('-', np.nan)
                prices['last'] = pd.to_numeric(prices['last'].str.replace(',', ''))
                prices = prices[['strike', 'type', 'last', 'settle', 'volume', 'openInterest']]
                prices = prices.rename(columns= {'openInterest': 'open_interest'})

                table[name][label]['trade_date'] = td
                table[name][label]['option_chain'] = prices
    return table

In [0]:
# Get list of uniques trade dates
def get_trade_dates(table: dict) -> list:
    td = list()
    for key in table.keys():
        for label in table[key]:
            td.append(table[key][label]['trade_date'])
    return sorted(list(set(td)))

def get_futures_settlement_prices(trade_date: dt.date, proxies: dict=None) -> pd.DataFrame:
    url = ''.join(
        (
            'https://www.cmegroup.com/CmeWS/mvc/Settlements/Futures/Settlements/133/FUT?', 
            'tradeDate={:02d}/{:02d}/{:4d}&strategy=DEFAULT&pageSize=50'.format(
                trade_date.month, trade_date.day, trade_date.year)
        )
    )
    page = requests.get(url, proxies=proxies if proxies is not None else dict(), verify=False)
    
    df = pd.DataFrame(page.json()['settlements'])
    df = df[df['month'] != 'Total']
    df = df[['month', 'last', 'settle', 'volume', 'openInterest']]
    df['last'] = df['last'].str.lower().str.replace('a', '').str.replace('b', '').str.replace(',', '')
    df['last'] = pd.to_numeric(df['last'])
    df['settle'] = df['settle'].str.strip().str.replace(',', '')
    df['settle'] = pd.to_numeric(df['settle'])
    df['volume'] = df['volume'].str.strip().str.replace(',', '')
    df['volume'] = pd.to_numeric(df['volume'])
    df['openInterest'] = df['openInterest'].str.strip().str.replace(',', '')
    df['openInterest'] = pd.to_numeric(df['openInterest'])
    df['month'] = df['month'].apply(str_date_to_datetime)
    df = df.rename(columns={'openInterest': 'open_interest', 'month': 'expiry'})
    
    return df

def get_contracts_table(trade_dates: list, proxies: dict=None) -> dict:
    futures = dict()
    for trade_date in get_trade_dates(options_groups):
        print(trade_date)
        futures[trade_date] = get_futures_settlement_prices(trade_date, ersel_proxies())
    return futures

In [0]:
def ersel_proxies():
  return dict()

In [19]:
%%time
options_groups = get_options_details(ersel_proxies())
trade_dates = get_trade_dates(options_groups)

CPU times: user 16 ms, sys: 1.12 ms, total: 17.1 ms
Wall time: 363 ms


In [29]:
option_types = list(options_groups.keys())
option_types

['E-mini S&P 500 Options',
 'E-mini S&P 500 EOM Options',
 'E-mini S&P 500 Monday Weekly Options',
 'E-mini S&P 500 Weekly Options',
 'E-mini S&P 500 Wednesday Weekly Options']

In [0]:
option_dates = dict{
  'E-mini S&P 500 Options',
  'E-mini S&P 500 EOM Options',
  'E-mini S&P 500 Monday Weekly Options',
  'E-mini S&P 500 Weekly Options',
  'E-mini S&P 500 Wednesday Weekly Options',
}

In [0]:
def _get_year_month_from_str(x: str):
  x = ' '.join(x.strip().lower().split())
  month, year = x.split(' ')
  month = month_str_to_num(month)
  year = int(year)
  return year, month

def _get_year_month_week_from_str(x: str):
  x = ' '.join(x.strip().lower().split())
  _, date, year = x.split(' ')
  week, month = date.split('-')
  week = int(week)
  month = month_str_to_num(month)
  year = int(year)
  return year, month, week

In [0]:
for option_type in options_groups.keys():
  if option_type == 'E-mini S&P 500 Options':
    for expiry_label in options_groups[option_type].keys():
      year, month = _get_year_month_from_str(expiry_label)
      options_groups[option_type][expiry_label]['expiry'] = third_friday_of_month(year, month)
      #print(expiry_label, third_friday_of_month(year, month))
  elif option_type == 'E-mini S&P 500 EOM Options':
    for expiry_label in options_groups[option_type].keys():
      year, month = _get_year_month_from_str(expiry_label)
      options_groups[option_type][expiry_label]['expiry'] = last_business_day_of_month(year, month)
      #print(expiry_label, last_business_day_of_month(year, month))
  elif option_type == 'E-mini S&P 500 Monday Weekly Options':
    for expiry_label in options_groups[option_type].keys():
      year, month, week = _get_year_month_week_from_str(expiry_label)
      options_groups[option_type][expiry_label]['expiry'] = monday_of_week(year, month, week)
      #print(expiry_label, monday_of_week(year, month, week))
  elif option_type == 'E-mini S&P 500 Weekly Options':
    for expiry_label in options_groups[option_type].keys():
      year, month, week = _get_year_month_week_from_str(expiry_label)
      options_groups[option_type][expiry_label]['expiry'] = friday_of_week(year, month, week)
      #print(expiry_label, friday_of_week(year, month, week))
  elif option_type == 'E-mini S&P 500 Wednesday Weekly Options':
    for expiry_label in options_groups[option_type].keys():
      year, month, week = _get_year_month_week_from_str(expiry_label)
      options_groups[option_type][expiry_label]['expiry'] = wednesday_of_week(year, month, week)
      #print(expiry_label, wednesday_of_week(year, month, week))

In [50]:
%%time
options_chain = retrieve_options_chain(options_groups)

E-mini S&P 500 Options
    Mar 2020
    Jun 2020
    Sep 2020
    Dec 2020
E-mini S&P 500 EOM Options
    Jan 2020
    Feb 2020
    Mar 2020
    Apr 2020
    May 2020
    Jun 2020
E-mini S&P 500 Monday Weekly Options
    Week 3-Jan 2020
    Week 4-Jan 2020
    Week 1-Feb 2020
    Week 2-Feb 2020
    Week 3-Feb 2020
E-mini S&P 500 Weekly Options
    Week 4-Jan 2020
    Week 1-Feb 2020
    Week 2-Feb 2020
    Week 3-Feb 2020
    Week 1-Mar 2020
    Week 2-Mar 2020
    Week 3-Apr 2020
    Week 3-May 2020
E-mini S&P 500 Wednesday Weekly Options
    Week 4-Jan 2020
    Week 5-Jan 2020
    Week 1-Feb 2020
    Week 2-Feb 2020
    Week 3-Feb 2020
CPU times: user 514 ms, sys: 12.5 ms, total: 526 ms
Wall time: 4.98 s


In [107]:
futures = get_contracts_table(get_trade_dates(options_groups), ersel_proxies())

2020-01-21
2020-01-22
2020-01-24
2020-01-27


In [0]:
output = list()
for trade_date, df in futures.items():
  df_ = df.copy()
  df_['trade_date'] = trade_date
  output.append(df_)
output = pd.concat(output, axis='index', sort=False)
output.to_csv('./futures.csv')

In [0]:
output = list()
for opt_type in options_chain.keys():
  for expiry_label in options_chain[opt_type].keys():
    trade_date = options_chain[opt_type][expiry_label]['trade_date']
    expiry_date = options_chain[opt_type][expiry_label]['expiry']
    chain = options_chain[opt_type][expiry_label]['option_chain'].copy()
    chain['trade_date'] = trade_date
    chain['expiry'] = expiry_date
    chain['opt_type'] = opt_type
    output.append(chain)
output = pd.concat(output, axis='index', sort=False)    
output.to_csv('./options.csv')

In [0]:
from google.colab import files
files.download("./futures.csv")
files.download("./options.csv")

In [100]:
import smtplib

server = smtplib.SMTP('smtp.mail.yahoo.com', 587)
server.set_debuglevel(1)
server.ehlo()
server.starttls()
server.ehlo
server.login("power_teo@yahoo.fr", "Colab1942")

msg = "COLAB WORK FINISH ALERT!"
server.sendmail("power_teo@yahoo.fr", "matteo.pugnaloni@ersel.it", msg)
server.quit()

send: 'ehlo [172.28.0.2]\r\n'
reply: b'250-smtp425.mail.gq1.yahoo.com Hello [172.28.0.2] [35.230.38.223])\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-SIZE 41697280\r\n'
reply: b'250 STARTTLS\r\n'
reply: retcode (250); Msg: b'smtp425.mail.gq1.yahoo.com Hello [172.28.0.2] [35.230.38.223])\nPIPELINING\nENHANCEDSTATUSCODES\n8BITMIME\nSIZE 41697280\nSTARTTLS'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [172.28.0.2]\r\n'
reply: b'250-smtp425.mail.gq1.yahoo.com Hello [172.28.0.2] [35.230.38.223])\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-SIZE 41697280\r\n'
reply: b'250 AUTH PLAIN LOGIN XOAUTH2 OAUTHBEARER\r\n'
reply: retcode (250); Msg: b'smtp425.mail.gq1.yahoo.com Hello [172.28.0.2] [35.230.38.223])\nPIPELINING\nENHANCEDSTATUSCODES\n8BITMIME\nSIZE 41697280\nAUTH PLAIN LO

SMTPServerDisconnected: ignored